In [0]:
%sql
/*
NoteBook que simula el Jobs 
en un solo NoteBook el subproceso Devolución de Pago sin Justificación Legal
y genera el archivo de salida

• SUBPROCESO 3832
*/

In [0]:
%run "./startup" 

In [0]:
params = WidgetParams(
    {
        "sr_proceso": str,
        "sr_subproceso": str,
        "sr_subetapa": str,
        "sr_folio": str,
        "sr_instancia_proceso": str,
        "sr_usuario": str,
        "sr_etapa": str,
        "sr_id_snapshot": str,
        "sr_id_archivo": str,
        "sr_fec_arc": str,
        "sr_paso": str,
        "sr_id_archivo_06": str,
        "sr_id_archivo_09": str
    }
)
params.validate()
conf = ConfManager()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columan 'Archivo SQL' comiencen por el nombre del notebook
display(queries_df.filter(col("Archivo SQL").startswith("GAR")))

In [0]:
statement_001 = query.get_statement(
"GAR_0300_SUM09_DPSJL.sql",
SR_FOLIO = params.sr_folio,
SR_SUBPROCESO = params.sr_subproceso,
)

In [0]:

if conf.debug:
    df = db.read_data("default", statement_001)
if conf.debug:
    display(df)
    display(str(df.count()) + " Registros Totales")

In [0]:

#Temp delta Sumario
temp_delta_sumario = 'DELTA_0300_SUM09_DPSJL' + '_' + params.sr_folio
db.write_delta(temp_delta_sumario, db.read_data("default",statement_001) , "overwrite")

if conf.debug:
    display(db.read_delta(temp_delta_sumario))

In [0]:
statement_002_final = query.get_statement(
"GAR_0100_GEN_ARCHIVO_RESP_DPSJL.sql",
DELTA_DPSJL_ENCABEZADO = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.DELTA_0100_ENCAB_DPSJL_{params.sr_folio}",
DELTA_DPSJL_DETALLE = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.DELTA_0200_DET02_DPSJL_{params.sr_folio}",
DELTA_DPSJL_SUMARIO = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.DELTA_0300_SUM09_DPSJL_{params.sr_folio}"
)

In [0]:
df_final = spark.sql(statement_002_final) 
if conf.debug:
    display(df_final)
    display(str(df_final.count()) + " Registros Totales")

In [0]:
statement_002 = query.get_statement(
    "GAR_0600_CONSEC_ARCHIVO.sql",
    SR_FOLIO=params.sr_folio,
)

In [0]:
from datetime import datetime
import pytz

# Inicializa la clase para subir los archivos
file_manager = FileManager(err_repo_path=conf.err_repo_path)

# Define the timezone for Mexico
mexico_tz = pytz.timezone("America/Mexico_City")

# Genera la fecha actual en el formato YYYYMMDD
fecha_actual_archivo = datetime.now(mexico_tz).strftime("%Y%m%d")

# Genera la fecha actual en el formato DD-MM-YYYY
fecha_actual_bd = datetime.now(mexico_tz).strftime("%d-%m-%Y")

#fecha timeStamp
fecha_actual_timestamp = datetime.now(mexico_tz).strftime("%d-%m-%Y %H:%M:%S.%f")

# Genera Consecutivo de Archivo

df_archivo_consecutivo = db.read_data("default", statement_002)


#Toma del Dataframe solo el valor de la Columna FTN_ID_ARCHIVO generada con el statement_002
#id_consecutivo = df_archivo_consecutivo.collect()[0]["FTN_ID_ARCHIVO"]
id_consecutivo = df_archivo_consecutivo.first()["FTN_ID_ARCHIVO"]


# Crea el nombre completo del archivo concatenando la ubicación externa, el camino del repositorio de errores y la fecha actual
full_file_name = (
        conf.external_location
        + conf.err_repo_path
        + conf.output_file_name_dpsjl_1
        + fecha_actual_archivo
        + "_"
        + str(id_consecutivo)
        +conf.output_file_name_dpsjl_2
        + ".txt"
    )


if conf.debug:
    display(full_file_name)


In [0]:
# Genera el archivo y decide si calcular MD5 o no

file_manager.generar_archivo_flexible(
    df=df_final,
    full_file_name=full_file_name,
    opciones={
        "header": "false",
        "encoding": "ISO-8859-1",
        #"delimiter": "",
        "linesep": "\r\n",
        "quote": "",
        "charset": "latin1",
        "escapeQuotes": False,
        "ignoreLeadingWhiteSpace": "false",
        "ignoreTrailingWhiteSpace": "false"
    },
    calcular_md5=False
)


In [0]:
CleanUpManager.cleanup_notebook(locals())